# Snowflake + Pandas

How to load data from a Snowflake table or query into a pandas dataframe

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
    </tr>
</table>

## Connect to Snowflake

See [README](README.md) for more details on how to set up the credentials environment variables for SNOWFLAKE_ACCOUNT, SNOWFLAKE_USER, and SNOWFLAKE_PASSWORD.

The other variables can be set on your Jupyter server or overwritten below based on the Snowflake warehouse and schema you used when running `load-data.sql`. Note that in order to update environment variables your Jupyter server will need to be stopped.

In [ ]:
import os

SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]

SNOWFLAKE_WAREHOUSE = os.environ["SNOWFLAKE_WAREHOUSE"]
TAXI_DATABASE = os.environ["TAXI_DATABASE"]
TAXI_SCHEMA = os.environ["TAXI_SCHEMA"]

In [ ]:
import snowflake.connector

conn_info = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": TAXI_DATABASE,
    "schema": TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

## Run query

The [Snowflake Connector for Python](https://docs.snowflake.com/en/user-guide/python-connector-pandas.html) has `fetch_pandas_all()` and `fetch_pandas_batches()` methods that utilize [Arrow](https://arrow.apache.org/) for fast data exchange.

In [ ]:
query = """
SELECT *
FROM taxi_yellow
WHERE
    date_trunc('DAY', pickup_datetime) = '2020-01-01'
"""
cur = conn.cursor().execute(query)
df = cur.fetch_pandas_all()

In [ ]:
df.head()

In [ ]:
len(df), df.memory_usage().sum() / 1e6  # memory size in MB

`fetch_pandas_batches()` is useful if you can perform operations if the full result doesn't fit in memory, but there are operations you can perform to individual batches. It returns a `generator` that you can loop over.

In [ ]:
cur = conn.cursor().execute(query)
batches = cur.fetch_pandas_batches()
batches

In [ ]:
for batch in batches:
    print(len(batch), batch.memory_usage().sum() / 1e6)

If the data and/or computation are just too big for pandas on a single node, that's when you move to Dask! Check out the [`snowflake-dask.ipynb`](snowflake-dask.ipynb) notebook for the Dask implementation of this example.

<hr>

## Next Steps

In this tutorial, you learned how to use Snowflake and `snowflake-connector-python` to execute SQL queries over large datasets. You also learned how to read those query results into a `pandas` data frame.

If you want to work with large query results or want to do post-processing that would benefit from a lot more parallelism than you can achieve on a single machine, you might want to read this result into a Dask DataFrame instead. Try [this dask + snowflake notebook](./snowflake-dask.ipynb) to learn how to efficiently read Snowflake query results into Dask collections.

If you want to see how to combine the lessons from this notebook with common machine learning tasks like feature engineering and hyperparameter tuning, try [this scikit-learn notebook](../nyc-taxi-snowflake/hyperparameter-scikit.ipynb).

<hr>